<a href="https://colab.research.google.com/github/ChiiKang/Python-Projects/blob/main/Text_Mining%20Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
d1 = "The sky is blue"
d2 = "The sun is bright"
d3 = "The sun in the sky is bright"
d4 = "We can see the shinning sun, the bright sun"
d5 = "Sun oh Mr. Sun, please shine on us. YOur lovely sunshine makes my day bright"
d6 = "Sun, Sun go away, I hate sun, sun sun sun sun sun!"

original_documents = [d1, d2, d3, d4, d5, d6]

In [ ]:
# Create function in python to practice reusability
def tokenize(data):  #def:define
  import re # Regular Expression
  stopwords = open("stopwords.txt","r").read().splitlines()  #r is the format, stopwords is a list

  #Tokenizeation the text pass into this function
  tokens = re.findall(r'\w+',data)   #\w: alpha numerical (0-9,a-z,)  +:take the whole length r:regular expression

  #Return the wordss in lower case where it doesn't appear in stopwords file
  return [t.lower() for t in tokens if t.lower() not in stopwords]

In [ ]:
documents = []
for document in original_documents:
  documents.append(tokenize(document))
documents

[['sky', 'blue'],
 ['sun', 'bright'],
 ['sun', 'sky', 'bright'],
 ['shinning', 'sun', 'bright', 'sun'],
 ['sun', 'sun', 'shine', 'lovely', 'sunshine', 'makes', 'day', 'bright'],
 ['sun', 'sun', 'hate', 'sun', 'sun', 'sun', 'sun', 'sun', 'sun']]

In [ ]:
from collections import Counter
from math import log

In [ ]:
# Term Frequency
term_tf = []
for document in documents:
  length = len(document)
  #Counter(document).items will appear (term,frequency), the front part is to calculate
  term_frequency = [(term,frequency/float(length))for term,frequency in Counter(document).items()]
  term_tf.append(term_frequency)
term_tf

[[('sky', 0.5), ('blue', 0.5)],
 [('sun', 0.5), ('bright', 0.5)],
 [('sun', 0.3333333333333333),
  ('sky', 0.3333333333333333),
  ('bright', 0.3333333333333333)],
 [('shinning', 0.25), ('sun', 0.5), ('bright', 0.25)],
 [('sun', 0.25),
  ('shine', 0.125),
  ('lovely', 0.125),
  ('sunshine', 0.125),
  ('makes', 0.125),
  ('day', 0.125),
  ('bright', 0.125)],
 [('sun', 0.8888888888888888), ('hate', 0.1111111111111111)]]

In [ ]:
#Document Freuency
N = len(documents)
#{} initialize dictionary
df = {}
#Loop for documents
for document in documents:
  #Loop for term
  for term in document:
      if term in df:
        #if term exist in df dictionary
        #past this round
        continue
      else:
        #if term doesn't exist in df dictionary
        #create a new key and set the counter to Zero(0)
          df[term]=0 #Initialize the counter

      #Loop documents again
      for doc in documents:
        #check if term exist in the document
        if term in doc:
          #Add 1 to the term if it exist in the document
          df[term] = df[term]+ 1. #Dont forget the DOT
df
  #blue occur in 1 document, bridge occur in 4 documents...

{'blue': 1.0,
 'bright': 4.0,
 'day': 1.0,
 'hate': 1.0,
 'lovely': 1.0,
 'makes': 1.0,
 'shine': 1.0,
 'shinning': 1.0,
 'sky': 2.0,
 'sun': 5.0,
 'sunshine': 1.0}

In [ ]:
# Inverse document frequency
# Create a dictionary variable
idf = {}
for term in df:
  idf[term]=log(N/df[term])
idf

{'blue': 1.791759469228055,
 'bright': 0.4054651081081644,
 'day': 1.791759469228055,
 'hate': 1.791759469228055,
 'lovely': 1.791759469228055,
 'makes': 1.791759469228055,
 'shine': 1.791759469228055,
 'shinning': 1.791759469228055,
 'sky': 1.0986122886681098,
 'sun': 0.1823215567939546,
 'sunshine': 1.791759469228055}

In [ ]:
# TFiDF
features = []
for term_freq in term_tf:
  tfidf = [(term,f*idf[term]) for term,f in term_freq]
  features.append(tfidf)
features

[[('sky', 0.5493061443340549), ('blue', 0.8958797346140275)],
 [('sun', 0.0911607783969773), ('bright', 0.2027325540540822)],
 [('sun', 0.060773852264651526),
  ('sky', 0.3662040962227032),
  ('bright', 0.13515503603605478)],
 [('shinning', 0.44793986730701374),
  ('sun', 0.0911607783969773),
  ('bright', 0.1013662770270411)],
 [('sun', 0.04558038919848865),
  ('shine', 0.22396993365350687),
  ('lovely', 0.22396993365350687),
  ('sunshine', 0.22396993365350687),
  ('makes', 0.22396993365350687),
  ('day', 0.22396993365350687),
  ('bright', 0.05068313851352055)],
 [('sun', 0.16206360603907075), ('hate', 0.19908438546978388)]]

In [ ]:
import re
import math
def cosim(vec1, vec2):
    v1 = dict(vec1)
    v2 = dict(vec2)

    intersection = set(v1.keys()) & set(v2.keys())
    numerator = sum([v1[x] * v1[x] for x in intersection])

    sum1 = sum([v1[x]**2 for x in v1.keys()])
    sum2 = sum([v2[x]**2 for x in v2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [ ]:
import operator
#keyword query
query = 'sun is bright'
#Tokenize the keyword query
q = [(token,1) for token in re.findall('\w+', query)]
print(q)
results = {}
#Calculate the cosimilarity of the keyword query
#with the features/tfidf
for i in range(len(features)):
    results[i] = cosim(features[i],q)
#sort the results based on the cosimilarity result, reverse:Descending
sorted_results = sorted(results.items(),key=operator.itemgetter(1), reverse = True)

for (i, score) in sorted_results:
    print("{0} - {1}".format(score,original_documents[i]))

[('sun', 1), ('is', 1), ('bright', 1)]
0.12833650557711443 - The sun is bright
0.05907045026551179 - Sun, Sun go away, I hate sun, sun sun sun sun sun!
0.032094061350424696 - The sun in the sky is bright
0.02291690881451253 - We can see the shinning sun, the bright sun
0.00530751106274252 - Sun oh Mr. Sun, please shine on us. YOur lovely sunshine makes my day bright
0.0 - The sky is blue
